# Analyze based on semantic categories

1.) change tfidf so we compare equivalent categories only - done
2.) update ranking accordingly

In [1]:
import os
from collections import Counter, defaultdict
import csv
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)


def get_properties():
    properties = []
    for path in os.listdir('../data/aggregated/'):
        prop = path.split('.')[0]
        if 'female-' not in prop and prop != '':
            properties.append(prop)
    return properties

In [3]:
# check updated corpus extraction
f_original = os.listdir('../contexts/giga_full/vocab')
print(len(f_original))
f_update = os.listdir('../contexts/giga_full_updated/vocab')
print(len(f_update))

1446
1636


## Show top distinctive contexts per property

* calculate mean f1 per sematic category
* rank contexts by mean f1
* only include categories with at least 10 pos and 10 neg examples (small categories distort mean)

In [9]:
pd.set_option('display.max_colwidth', None)
# get top distinctive contexts per prop
import get_top_contexts_distinctiveness

model_name = 'giga_full_updated'
properties = get_properties()
table = get_top_contexts_distinctiveness.get_top_distinctive_contexts(properties, model_name)
df = pd.DataFrame(table)
df.sort_values('f1-mean', ascending = False).round(2)

,property,n_contexts,f1-mean,contexts
9,used_in_cooking,572,0.94,add recipe fish
4,fly,952,0.90,payload study experimental hovering flew overhead
11,green,578,0.86,place shade belongs green citrus
0,square,1504,0.85,built
13,blue,2234,0.85,magic bright scale various european
16,female,45,0.84,actress birth herself
20,lay_eggs,74,0.83,eggs
8,hot,84,0.83,flame oven remove hot
7,sweet,35,0.83,banana potato sweet
14,yellow,52,0.83,sipping operating apple yellow good bubbly apples


In [6]:
# latex table for paper:
cols = ['property', 'n_contexts', 'f1-mean', 'contexts']
df = df.sort_values('f1-mean', ascending = False).round(2)
print(df[cols].to_latex(index=False))

\begin{tabular}{lrrl}
\toprule
        property &  n\_contexts &  f1-mean &                                           contexts \\
\midrule
 used\_in\_cooking &         572 &     0.94 &                                    add recipe fish \\
             fly &         952 &     0.90 &  payload study experimental hovering flew overhead \\
           green &         578 &     0.86 &                   place shade belongs green citrus \\
          square &        1504 &     0.85 &                                              built \\
            blue &        2234 &     0.85 &                magic bright scale various european \\
          female &          45 &     0.84 &                              actress birth herself \\
        lay\_eggs &          74 &     0.83 &                                               eggs \\
             hot &          84 &     0.83 &                              flame oven remove hot \\
           sweet &          35 &     0.83 &                               

In [10]:
# get top distinctive contexts per prop

model_name = 'wiki_updated'
properties = get_properties()
table = get_top_contexts_distinctiveness.get_top_distinctive_contexts(properties, model_name)
df = pd.DataFrame(table)
df.sort_values('f1-mean', ascending = False).round(2)

,property,n_contexts,f1-mean,contexts
9,used_in_cooking,705,0.92,meat food
16,female,109,0.90,la female birth she
4,fly,55,0.90,thrust fly demonstrate bird experimental flown owl
14,yellow,111,0.88,name whiskey beer tribe half
6,wings,332,0.87,birds
13,blue,1819,0.87,simple abundant evening
2,black,820,0.86,north any size black accidentally typical led
19,wheels,105,0.85,chassis driver
11,green,550,0.85,fruits flora green found citrus forest
5,dangerous,627,0.84,hand killing


In [8]:
# latex table for paper:
cols = ['property', 'n_contexts', 'f1-mean', 'contexts']
df = df.sort_values('f1-mean', ascending = False).round(2)
print(df[cols].to_latex(index=False))

\begin{tabular}{lrrl}
\toprule
        property &  n\_contexts &  f1-mean &                                           contexts \\
\midrule
 used\_in\_cooking &         705 &     0.92 &                                          meat food \\
          female &         109 &     0.90 &                                la female birth she \\
             fly &          55 &     0.90 &  thrust fly demonstrate bird experimental flown... \\
          yellow &         111 &     0.88 &                       name whiskey beer tribe half \\
           wings &         332 &     0.87 &                                              birds \\
            blue &        1819 &     0.87 &                            simple abundant evening \\
           black &         820 &     0.86 &      north any size black accidentally typical led \\
          wheels &         105 &     0.85 &                                     chassis driver \\
           green &         550 &     0.85 &             fruits flora green 

## Prepare context annotation

In [ ]:
# import prepare_annotations

# model_name = 'giga_full_updated'
# properties = get_properties()
# #properties_test = ['dangerous', 'cold', 'lay_eggs']
# #properties = [p for p in properties if p not in properties_test]
# #properties = properties_test
# properties = ['dangerous']
# cutoff = 3
# cutoff_concepts = 3

# for prop in properties:
#     print(prop)
    
#     prepare_annotations.aggregate_contexts(prop, cutoff, model_name)
#     prepare_annotations.prepare_annotation(prop, model_name, cutoff, cutoff_concepts)

### Transfer giga to wiki annotations 


In [11]:
# run: python transfer_annotations.py

### Complete annotations

In [23]:
import analyze_annotations
import numpy as np
import pandas as pd

In [29]:
model_name = 'wiki_updated'
top_cutoff = 3
concept_cutoff = 3
ann_dict = analyze_annotations.show_annotation_status(model_name, top_cutoff, concept_cutoff)

completed:

black (821, 634, 187)
blue (1820, 1018, 802)
cold (591, 382, 209)
dangerous (628, 392, 236)
female (110, 97, 13)
fly (56, 41, 15)
green (551, 402, 149)
hot (93, 78, 15)
juicy (123, 59, 64)
lay_eggs (27, 18, 9)
made_of_wood (498, 373, 125)
yellow (112, 108, 4)

Incomplete:

red (1276, 781, 495)
roll (3486, 1464, 2022)
round (1035, 886, 149)
square (443, 214, 229)
sweet (27, 20, 7)
swim (1701, 1225, 476)
used_in_cooking (706, 492, 214)
warm (1182, 573, 609)
wheels (106, 66, 40)
wings (333, 234, 99)


### Consistency checks

1.) Consistency between the corpora

2.) Property words

3.) Instances that are positive examples 

4.) Colors: other colors should count as related?


5.) Sample instances and let them be double-annotated?

### Evidence density

*How many of the evidence candidates are actual property evidence? How are they distributed over differnt types of evidence?*

In [18]:
model_name = 'wiki_updated'
top_cutoff = 3
concept_cutoff = 3
ann_dict = analyze_annotations.show_annotation_status(model_name, top_cutoff, concept_cutoff)
properties = ann_dict['complete']


cols = ['property', 'u', 'all', 'prop_specific', 'non-specific', 'p', 'n', 'l', 'i', 'r', 'b']
table = []
for prop in properties:
    d = dict()
    d['property'] =  prop
    d.update(analyze_annotations.get_evidence_density(model_name, prop, top_cutoff, concept_cutoff))
    for c in cols:
        if c not in d:
            d[c] = np.nan
    table.append(d)
   
cols = ['property', 'u', 'all', 'prop_specific', 'non-specific', 'p', 'n', 'l', 'i', 'r', 'b']
df = pd.DataFrame(table)[cols]
df = df[cols].sort_values('all', ascending = False).round(3)
df

completed:

black (821, 634, 187)
blue (1820, 1018, 802)
cold (591, 382, 209)
dangerous (628, 392, 236)
female (110, 97, 13)
fly (56, 41, 15)
green (551, 402, 149)

Incomplete:

hot (93, 78, 15)
juicy (123, 59, 64)
lay_eggs (27, 18, 9)
made_of_wood (498, 373, 125)
red (1276, 781, 495)
roll (3486, 1464, 2022)
round (1035, 886, 149)
square (443, 214, 229)
sweet (27, 20, 7)
swim (1701, 1225, 476)
used_in_cooking (706, 492, 214)
warm (1182, 573, 609)
wheels (106, 66, 40)
wings (333, 234, 99)
yellow (112, 108, 4)


,property,u,all,prop_specific,non-specific,p,n,l,i,r,b
1,fly,0.582,0.418,0.091,0.327,0.073,NaN,0.018,0.164,0.164,NaN
3,female,0.798,0.202,0.028,0.174,0.009,NaN,0.018,0.128,NaN,0.046
4,green,0.864,0.136,0.002,0.135,0.002,NaN,NaN,0.120,0.015,NaN
6,dangerous,0.884,0.116,0.010,0.107,0.002,0.002,0.006,0.051,0.056,NaN
0,blue,0.921,0.079,0.001,0.078,0.001,NaN,NaN,0.073,0.004,NaN
5,black,0.927,0.073,0.005,0.067,0.001,0.001,0.002,0.061,0.006,NaN
2,cold,0.956,0.044,0.003,0.041,0.002,0.002,NaN,0.036,0.005,NaN


In [6]:
#print(df.round(3).fillna('-').to_latex(index=False))

### Evidence diversity

*How many different words are used to express property evidence?*

In [14]:
model_name = 'giga_full_updated'
top_cutoff = 3
concept_cutoff = 3
ann_dict = analyze_annotations.show_annotation_status(model_name, top_cutoff, concept_cutoff)
properties = ann_dict['complete']
top_cutoff = 3
concept_cutoff = 3

cols = ['property', 'u', 'all', 'prop_specific', 'non-specific', 'p', 'n', 'l', 'i', 'r', 'b']
table = []
for prop in properties:
    d = dict()
    d['property'] =  prop
    d.update(analyze_annotations.get_evidence_diversity(model_name, prop, top_cutoff, concept_cutoff))
    for c in cols:
        if c not in d:
            d[c] = np.nan
    table.append(d)
   
cols = ['property', 'u', 'all', 'prop_specific', 'non-specific', 'p', 'n', 'l', 'i', 'r', 'b']
df = pd.DataFrame(table)[cols]
df = df[cols].sort_values('all', ascending = False)
df

completed:

black (1150, 0, 1150)
blue (2235, 0, 2235)
cold (1082, 0, 1082)
dangerous (1115, 0, 1115)
female (46, 0, 46)
fly (953, 0, 953)
green (579, 0, 579)
hot (85, 0, 85)
juicy (811, 0, 811)
lay_eggs (75, 0, 75)
made_of_wood (787, 0, 787)
red (1768, 0, 1768)
roll (3887, 0, 3887)
round (521, 0, 521)
square (1505, 0, 1505)
sweet (36, 0, 36)
swim (1382, 0, 1382)
used_in_cooking (573, 0, 573)
warm (2109, 0, 2109)
wheels (245, 0, 245)
wings (561, 0, 561)
yellow (53, 0, 53)

Incomplete:



,property,u,all,prop_specific,non-specific,p,n,l,i,r,b
18,used_in_cooking,204,368,8.0,360,4.0,2.0,2.0,173,187.0,NaN
21,blue,2066,168,2.0,166,2.0,NaN,NaN,156,10.0,NaN
1,roll,3741,144,9.0,135,5.0,NaN,4.0,91,44.0,NaN
19,dangerous,984,130,23.0,107,2.0,7.0,14.0,41,60.0,6.0
15,warm,1998,109,8.0,101,4.0,1.0,3.0,79,22.0,NaN
8,juicy,713,97,2.0,95,2.0,NaN,NaN,83,12.0,NaN
17,red,1673,94,1.0,93,1.0,NaN,NaN,87,6.0,NaN
14,green,485,93,1.0,92,1.0,NaN,NaN,84,8.0,NaN
13,wings,476,84,6.0,78,1.0,NaN,5.0,25,38.0,15.0
9,cold,1012,69,6.0,62,1.0,5.0,NaN,43,19.0,NaN


### Evidence distinctiveness

In [16]:
# import json
# import csv
# import os
# from collections import Counter, defaultdict
import analyze_evidence

import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [17]:
prop = 'female'
model_name = 'giga_full_updated'
top_cutoff = 3
concept_cutoff = 3
table = analyze_evidence.get_top_ev_categories(prop, model_name, top_cutoff, concept_cutoff)
df = pd.DataFrame(table)
df.T

f1                  contexts  pos  neg
all    l 0.69                  pregnant  109  144
       i 0.83                   herself  109  144
       b 0.80                 beautiful  109  144
       u 0.69                   stories  109  144
mammal l 0.86                     birth   10   12
       i 0.81  lady herself actress her   10   12
       b 0.63                      baby   10   12
       u 0.81                     grass   10   12
object l 0.70                  pregnant  104  142
       i 0.83                   herself  104  142
       b 0.80                 beautiful  104  142
       u 0.67                   stories  104  142
animal l 0.61                  pregnant   16   17
       i 0.84                   actress   16   17
       b 0.62                       sex   16   17
       u 0.81                   stories   16   17

In [3]:
#print(df.T.to_latex())

In [18]:
model_name = 'giga_full_updated'
top_cutoff = 3
concept_cutoff = 3

pd.options.display.float_format = "{:,.2f}".format

properties = get_properties()
prop_table = []
for prop in properties:
    table = analyze_evidence.get_top_ev_categories(prop, model_name, top_cutoff, concept_cutoff)
    type_scores = defaultdict(list)
    ev_type_mean_scores = dict()
    for (cat, ev_type), f1_dict in table.items():
        type_scores[ev_type].append(f1_dict['f1'])
    for ev_type, scores in type_scores.items():
        mean = sum(scores)/len(scores)
        prop_dict = dict()
        ev_type_mean_scores[ev_type] = mean
    prop_dict['property'] = prop
    prop_dict.update(ev_type_mean_scores)
    prop_table.append(prop_dict)
    
cols = ['property', 'p', 'n', 'l', 'i', 'r', 'b', 'u']
df = pd.DataFrame(prop_table)
df = df[cols].sort_values('p', ascending = False).fillna('-')#round(2).fillna('-')
df

,property,p,n,l,i,r,b,u
9,used_in_cooking,0.90,0.73,0.91,0.92,0.94,-,0.86
4,fly,0.89,0.80,0.78,0.85,0.85,0.68,0.84
20,lay_eggs,0.83,-,-,0.82,0.77,-,0.78
13,blue,0.81,-,-,0.81,0.83,-,0.84
19,wheels,0.81,-,0.70,0.80,0.82,-,0.71
7,sweet,0.81,-,-,0.81,-,-,0.74
11,green,0.78,-,-,0.79,0.77,-,0.83
14,yellow,0.78,-,-,0.78,-,-,0.81
10,juicy,0.76,-,-,0.81,0.77,-,0.79
2,black,0.75,-,0.56,0.74,0.67,-,0.83


In [19]:
#print(df.to_latex(index=False))

## Evidence strength


In [9]:
import analyze_evidence


def get_tfidf_scores_context(prop, cat, contexts, model_name):
    
    # collect scores here:
    context_tfidf_dict = defaultdict(dict)

    # get tfidf scores
    path_tfidfs = f'../results/{model_name}/tfidf-raw-10000/each_target_vs_corpus_per_category'
    paths_tfidf = [f'{path_tfidfs}/{prop}/{cat}/pos', f'{path_tfidfs}/{prop}/{cat}/neg']
    concept_files = []
    for path in paths_tfidf:
        concept_files.extend([f'{path}/{f}' for f in os.listdir(path) if f.endswith('.csv')])
    for cf in concept_files:
        full_path = cf
        #full_path = f'{path_tfidfs}/{cf}'
        concept = os.path.basename(cf).split('.')[0]
        with open(full_path) as infile:
            data = list(csv.DictReader(infile))
        for cw_target in contexts:
            found = False
            for d in data:
                cw = d['']
                score = d['target']
                if cw == cw_target:
                    context_tfidf_dict[cw][concept] = (float(score))
                    found = True
                    break
            if found == False:
                context_tfidf_dict[cw_target][concept]  = 0.0
    return context_tfidf_dict


def evidence_strength_to_file(prop, et, contexts, model_name, top_cutoff, concept_cutoff):

    filepath_target = f'../analysis/{model_name}/evidence_{top_cutoff}_{concept_cutoff}-raw-10000-categories'
    filepath_target_et = f'{filepath_target}/{prop}/{et}'
    os.makedirs(filepath_target_et, exist_ok=True)
    categories = analyze_evidence.get_categories(prop, model_name)
    tfidf_scores_cats = dict()
              
    for context in contexts:
        full_filepath = f'{filepath_target_et}/{context}.csv'
        for cat in categories:
            tfidf_scores_cats[cat] = get_tfidf_scores_context(prop, cat, contexts, model_name)[context]
            #print(cat, tfidf_scores_cats[cat]['wren'])
        df = pd.DataFrame(tfidf_scores_cats)
        df.fillna(0, inplace=True)
        df['mean'] = df.mean(numeric_only=True, axis = 1)
        df.to_csv(full_filepath)

In [11]:

# extract mean evidence strength for context words that give prop-specific evidence
model_name = 'giga_full_updated'
properties = get_properties()

#properties = ['dangerous']

top_cutoff = 3
concept_cutoff = 3


for prop in properties:
    print(prop)
    table = analyze_evidence.get_top_ev_categories(prop, model_name, top_cutoff, concept_cutoff)

    ets = ['p', 'n', 'l']
    for et_target in ets:
        contexts = set()
        for (cat, et), d in table.items():
            if et_target == et:
                contexts.update(set(d['contexts'].split(' ')))
        if len(contexts) > 0:
            evidence_strength_to_file(prop, et_target, contexts, model_name, top_cutoff, concept_cutoff)

square
warm
black
red
fly
dangerous
wings
sweet
hot
used_in_cooking
juicy
green
made_of_wood
blue
yellow
roll
female
cold
round
wheels
lay_eggs
swim


In [12]:
# compare evidence strength across properties - may not make too much sense...

ets = ['p', 'l', 'n']
properties = get_properties()

model_name = 'giga_full_updated'

top_cutoff = 3
concept_cutoff = 3

prop_et_strength_dict = defaultdict(dict)

path_evidence = f'../analysis/{model_name}/evidence_{top_cutoff}_{concept_cutoff}-raw-10000-categories'

for prop in properties:
    means_prop = []
    for et in ets:
        means_et = []
        path_dir = f'{path_evidence}/{prop}/{et}'
        if os.path.isdir(path_dir):
            ev_files = [f for f in os.listdir(path_dir) if f.endswith('.csv')]
            for f in ev_files:
                full_path = f'{path_dir}/{f}'
                with open(full_path) as infile:
                    data = list(csv.DictReader(infile))
                    means = [float(d['mean']) for d in data]
                    if len(means) > 0:
                        mean = sum(means)/len(means)
                    else:
                        mean = 0.0
                    #means_et.append(mean)
                    means_prop.append(mean)
    if len(means_prop) > 0:
        mean_prop = sum(means_prop)/len(means_prop)
    else:
        mean_prop = 0.0
    prop_et_strength_dict[prop]['prop_specific'] = mean_prop
            
df = pd.DataFrame(prop_et_strength_dict)
df.T.sort_values('prop_specific', ascending=False).round(4)

,prop_specific
juicy,0.0120
female,0.0113
red,0.0062
lay_eggs,0.0055
hot,0.0053
sweet,0.0051
made_of_wood,0.0048
wheels,0.0047
green,0.0041
yellow,0.0037


## Relation analysis

In [13]:
from statistics import stdev
import numpy as np
import analyze_evidence

In [14]:

  
def get_relation_combinations(properties, combinations):
    
    relation_pair_dict = defaultdict(set)
    for prop in properties:
        prop_dict = analyze_evidence.load_prop_data(prop)
        for c, d in prop_dict.items():
            ml_label = d['ml_label']
            if ml_label in {'all', 'some', 'all-some', 'few-some'}:
                l = 'pos'
            elif ml_label in {'few'}:
                l = 'neg'
            relation_pair_dict[l].add((prop, c))
            if l == 'pos':
                rel_dict = d['relations']
                for combination in combinations:
                    relations = set([rel for rel, p in rel_dict.items() if p > 0.5])
                    if combination == relations:
                        l_comb = tuple(sorted(relations))
                        relation_pair_dict[l_comb].add((prop, c))
    return relation_pair_dict   

In [15]:

# get prop concept pairs with a certain relation-configuration:
combinations = [
                    {'implied_category'},
                    {'variability_limited'},
                    {'variability_open'},
                    {'afforded_usual'},
                    {'afforded_usual', 'implied_category'},
                    {'afforded_unusual'},
                    {'implied_category', 'variability_limited'},
                    {'typical_of_property'}, 
                    {'typical_of_concept'},
                    {'typical_of_concept', 'typical_of_property'}
                    ]

properties = get_properties()
relation_pair_dict = get_relation_combinations(properties, combinations)
for rel, pairs in relation_pair_dict.items():
     print(rel, len(pairs)) 
# print()
# print(relation_pair_dict[('implied_category',  )]) 
# print()
# print(relation_pair_dict[('variability_limited',  )]) 
# print()
# print(relation_pair_dict[('implied_category', 'variability_limited', )])

print()
print(relation_pair_dict[('typical_of_concept',  )]) 
print()
print(relation_pair_dict[('typical_of_concept', 'typical_of_property' )]) 
print()
print(relation_pair_dict[('implied_category', 'variability_limited', )])
print()
print(relation_pair_dict[('variability_limited', )])


neg 1545
pos 2135
('implied_category', 'variability_limited') 21
('variability_open',) 175
('variability_limited',) 114
('implied_category',) 16
('typical_of_concept', 'typical_of_property') 4
('typical_of_concept',) 4
('afforded_unusual',) 20
('afforded_usual', 'implied_category') 18
('afforded_usual',) 3

{('juicy', 'meat'), ('made_of_wood', 'club'), ('fly', 'babbler'), ('wheels', 'wheel')}

{('red', 'watermelon'), ('sweet', 'desert'), ('green', 'jade'), ('red', 'tomato')}

{('square', 'laptop'), ('red', 'tongue'), ('sweet', 'breadfruit'), ('green', 'fenugreek'), ('made_of_wood', 'ladle'), ('fly', 'fowl'), ('made_of_wood', 'transom'), ('round', 'cabbage'), ('sweet', 'carrot'), ('round', 'onion'), ('round', 'sapodilla'), ('roll', 'cart'), ('round', 'nutlet'), ('round', 'gourd'), ('wings', 'beetle'), ('made_of_wood', 'girder'), ('round', 'lemon'), ('square', 'computer'), ('square', 'blackboard'), ('juicy', 'anjou'), ('round', 'pineapple')}

{('yellow', 'nectarine'), ('made_of_wood', 'b

In [16]:
# focus on prop-specific evidence types


def get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff):
    
    path_evidence = f'../analysis/{model_name}/evidence_{top_cutoff}_{concept_cutoff}-raw-10000-categories'
    tfidf_scores = []
    for et in ets:
        path_prop_et = f'{path_evidence}/{prop}/{et}'
        if os.path.isdir(path_prop_et):
            files = [f for f in os.listdir(path_prop_et) if f.endswith('.csv')]
            for f in files:
                full_path = f'{path_prop_et}/{f}'
                with open(full_path) as infile:
                    data = list(csv.DictReader(infile))
                for d in data:
                    if d[''] == concept:
                        tfidf = d['mean']
                        tfidf_scores.append(float(tfidf))
                        break
    if len(tfidf_scores) > 0:
        mean = sum(tfidf_scores)/len(tfidf_scores)
    else:
        mean = 0.0
    return mean
            

#### Sanity check: 

Positive relations should have a higher mean tfidf value than negative relations

In [17]:
combination = ('pos')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

2135
0.0045963338372104375


In [18]:
combination = ('neg')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

1545
0.0015040926458269597


### Step 1: Check isolated relations

* advantage: no interference from other relations
* disadvantage: small number of prop-concept pairs

### Step 2: Check effects of combinations

* impliedness + relation associated with prop mentions should lead to higher values
* typical_of_concept + relation associated with prop mentions should lead to higher values

#### Impliedness:

* implied_category should have a low mean tfidf value
* Observation: Clearly lower than mean pos value, closer to mean neg value

In [19]:
combination = ('implied_category', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

16
0.0012937158078214462


#### Variability

* variability_limited should have a high mean tfidf value
* variability_open should have a low mean tfidf value

* Observations:


In [20]:
combination = ('variability_limited', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

114
0.003956634603549042


In [21]:
combination = ('variability_open', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

175
0.0017967697773990463


#### Variability in combination with impliedness should overrule impliedness

In [23]:
combination = ('implied_category', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

16
0.0012937158078214462


In [24]:
combination = ('implied_category', 'variability_limited')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(pairs)
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

{('square', 'laptop'), ('red', 'tongue'), ('sweet', 'breadfruit'), ('green', 'fenugreek'), ('made_of_wood', 'ladle'), ('fly', 'fowl'), ('made_of_wood', 'transom'), ('round', 'cabbage'), ('sweet', 'carrot'), ('round', 'onion'), ('round', 'sapodilla'), ('roll', 'cart'), ('round', 'nutlet'), ('round', 'gourd'), ('wings', 'beetle'), ('made_of_wood', 'girder'), ('round', 'lemon'), ('square', 'computer'), ('square', 'blackboard'), ('juicy', 'anjou'), ('round', 'pineapple')}
21
0.001578340098725068


In [25]:
combination = ('variability_limited', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(pairs)

{('yellow', 'nectarine'), ('made_of_wood', 'bass'), ('juicy', 'corn'), ('green', 'serviceberry'), ('roll', 'paint'), ('made_of_wood', 'roof'), ('black', 'calf'), ('yellow', 'peach'), ('blue', 'shark'), ('round', 'banjo'), ('made_of_wood', 'railing'), ('made_of_wood', 'mop'), ('red', 'ambulance'), ('round', 'pecan'), ('black', 'millipede'), ('fly', 'craft'), ('made_of_wood', 'axe'), ('juicy', 'chicken'), ('blue', 'dogbane'), ('square', 'desk'), ('juicy', 'doughnut'), ('red', 'leafs'), ('green', 'strawberry'), ('juicy', 'bean'), ('sweet', 'beetroot'), ('juicy', 'ham'), ('blue', 'starling'), ('black', 'sapsucker'), ('black', 'bear'), ('black', 'sky'), ('fly', 'capsule'), ('yellow', 'daisy'), ('black', 'penguin'), ('made_of_wood', 'beam'), ('green', 'chutney'), ('black', 'truffle'), ('red', 'lentil'), ('black', 'human'), ('red', 'carnation'), ('juicy', 'date'), ('roll', 'mandrel'), ('red', 'ant'), ('used_in_cooking', 'cake'), ('juicy', 'beet'), ('square', 'footrest'), ('juicy', 'ginger'), 

In [26]:
combination = ('implied_category', 'variability_open')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

6
0.0016937395316195085


#### Typicality

* typical_of_property should have higher values
* typical_of_concept should have lower values
* problem: they are unlikely to occur independently from one another

In [27]:
combination = ('typical_of_property', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

0


ZeroDivisionError: division by zero

In [28]:
combination = ('typical_of_concept', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

4
0.007614649891454527


#### Affordedness

In [29]:
combination = ('afforded_usual', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

3
0.00927845165257774


In [30]:
combination = ('afforded_unusual', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

20
0.00039111772001438174


In [31]:
combination = ('affording_activity', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

4
0.006053307639432479


#### Affordedness relations + other relations

*affording_activity*

In [32]:
combination = ('affording_activity', 'implied_category')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
print(pairs)
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

8
{('made_of_wood', 'bamboo'), ('wheels', 'dumper'), ('warm', 'tunic'), ('wheels', 'snowplow'), ('hot', 'lightning'), ('wheels', 'equipage'), ('warm', 'calfskin'), ('wheels', 'airplane')}
0.004585447563000917


In [33]:
combination = ('affording_activity', 'typical_of_concept')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

5
0.004864331820770777


In [34]:
combination = ('affording_activity', 'implied_category', 'typical_of_concept')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

57
0.0021921960452967494


*afforded_usual*

In [138]:
combination = ('afforded_usual', 'implied_category')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

18
0.025247162010199574
